# Citations in public policies - feature selection i topic modeling


### Topic modeling za pomocą BERTopic

- Embeddingi zdań z modelu `all-distilroberta-v1`. Pozwola on na użycie tekstów do 512 tokenów.
- Modelowanie do 10 tematów.
- 8 słów na temat
- Parametr `diversity=0.3` określający minimalną różnorodnośc słów pomiędzy tematami.
- Odfiltrowane angielskich stopwords po embeddingu (co ważne).

In [ ]:
topic_model.get_topics()

*Wykresy w html...*

# Feature selection 
## Co zrobione:
* dorzucone 10 topiców do danych
* wyrzucone kolumny sumujące się do 0 lub 1 (na całym X, połączonym train i test)- powinno być odfiltrowane wcześniej. Zostaje 28k

Pierwsze podejście do feature selection:
1. SelectKBest do top 5000
2. RecursiveFeatureElimination do top 1000 ze step=10 (estimator LinearRegression)
3. RFE do top 400 ze step=1

Przebieg:
* Aż  z 10 topiców zostało
* Skrajny overfitting.
* Odrzucono część bardzo rzadko występujących autorów.

Drugie podejście
1. SelectKbest do top5000
2. RFE do top 3000 step 5
3. RFE do top 2000 step 1

Minimalnie lepiej ale wciąż źle.

In [ ]:
import gdown
import pickle as pkl
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#id='1i9VGVhDOdQztHCtSLWMVBiqIQYY_Fmz-'
#output='train_data.pkl'
#gdown.download(id=id, output=output, quiet=False)

In [ ]:
 with open("train_data.pkl", "rb") as f:
    object_train = pkl.load(f)
X_train = pd.DataFrame(object_train[0]).drop(columns=["abstract"])
y_train=pd.DataFrame(object_train[1])

In [ ]:
#id='1pZjfMG_12oEJ-VSsOKHM4n_kESfkwN35'
#output='test_data.pkl'
#gdown.download(id=id, output=output, quiet=False)

In [ ]:
with open("test_data.pkl", "rb") as f:
    object_test = pkl.load(f)
X_test = pd.DataFrame(object_test[0]).drop(["abstract"], axis=1)
y_test=pd.DataFrame(object_test[1])

In [ ]:
train_topics=pd.read_csv("train_topic_features.csv")
test_topics=pd.read_csv("test_topic_features.csv")
X_train=pd.concat([X_train, train_topics], axis=1)
X_test=pd.concat([X_test, test_topics], axis=1)

### Wyrzucenie kolumn sumujących się do 0

In [ ]:
X=pd.concat([X_train, X_test], axis=0)
X=X[X.columns[X.sum(axis=0) != 0]]
X=X[X.columns[X.sum(axis=0) != 1]]
X

In [ ]:
X_train=X_train[X.columns]
X_test=X_test[X.columns]

In [ ]:
num_authors=len([col for col in X_train if col.startswith('authors')])
num_instit=len([col for col in X_train if col.startswith('institutions')])
num_countries=len([col for col in X_train if col.startswith('countries')])
num_field=len([col for col in X_train if col.startswith('mag_')])
num_jour=len([col for col in X_train if col.startswith('jour')])
num_type=len([col for col in X_train if col.startswith('type_')])
num_abs=len([col for col in X_train if col.startswith('abstract_')])
num_topic=10

In [ ]:
plt.bar(["authors", "institution", "countries", "mag_field_of_study", "journal name", "type of publication", "tdidf", "topic"],
       [num_authors, num_instit, num_countries, num_field, num_jour, num_type, num_abs, num_topic])
plt.xticks(rotation="vertical");
plt.yscale("log")
plt.title("Number of features by category")

# Feature selection

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from matplotlib import pyplot as plt


### SelectKBest

In [ ]:
Kselector = SelectKBest(chi2, k=5000)

In [ ]:
Kselector.fit(X_train, y_train)
cols=Kselector.get_support(indices=True)
X_train_new=X_train.iloc[:,cols]

In [ ]:
X_train_new

## Recursive feature elimination z logistic regression jako estymatorem

In [ ]:
from sklearn.feature_selection import RFE
estimator = LogisticRegression() 
rfe_selector = RFE(estimator, n_features_to_select=1000, step=10, verbose=1)
rfe_selector = rfe_selector.fit(X_train_new, y_train) 

In [ ]:
X_train_newer=X_train_new.iloc[:, rfe_selector.support_]
rfe_selector = RFE(estimator, n_features_to_select=400, step=1, verbose=1)
rfe_selector = rfe_selector.fit(X_train_newer, y_train)

In [ ]:
X_train_final=X_train_newer.iloc[:, rfe_selector.support_]
X_train_final

In [ ]:
num_authors=len([col for col in X_train_final if col.startswith('authors')])
num_instit=len([col for col in X_train_final if col.startswith('institutions')])
num_countries=len([col for col in X_train_final if col.startswith('countries')])
num_field=len([col for col in X_train_final if col.startswith('mag_')])
num_jour=len([col for col in X_train_final if col.startswith('jour')])
num_type=len([col for col in X_train_final if col.startswith('type_')])
num_abs=len([col for col in X_train_final if col.startswith('abstract_')])
num_topic=5

In [ ]:
plt.bar(["authors", "institution", "countries", "mag_field_of_study", "journal name", "type of publication", "tdidf", "topic"],
       [num_authors, num_instit, num_countries, num_field, num_jour, num_type, num_abs, num_topic])
plt.xticks(rotation="vertical");
plt.title("Number of features by category after selection")

In [ ]:
[print(i) for i in X_train_final.columns]

In [ ]:
X_test_final=X_test[X_train_final.columns]